# Elasticsearchを利用したサンプル検索

Elasticsearchを利用した検索などを試すためのノートブックです。

## 接続先の設定

**ESHOST**に接続先を設定します。サンプルでは、ローカルネットワーク環境にあるElasticsearchに接続しています。
接続先の各種設定については公式ドキュメントをご覧ください。
https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html

**INDEX**が接続先のインデックス名です。

In [13]:
from elasticsearch import Elasticsearch
import json

ESHOST = "http://es:9200"
INDEX = "esci-products"

esclient = Elasticsearch(ESHOST)

## スキーマの確認

対象となるインデックスのスキーマを確認します。

In [14]:
import json
response = esclient.indices.get(index=INDEX, pretty=True)
print(json.dumps(response.body, indent=2))

{
  "esci-products": {
    "aliases": {},
    "mappings": {
      "properties": {
        "Index": {
          "type": "long"
        },
        "product_brand": {
          "type": "keyword"
        },
        "product_bullet_point": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }
        },
        "product_color": {
          "type": "keyword"
        },
        "product_description": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }
        },
        "product_id": {
          "type": "keywo

## データの確認

件数を確認する

In [17]:
response = esclient.count(
    index=INDEX
)
response

ObjectApiResponse({'count': 0, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})


検索してみる。

In [16]:
import pandas as pd

response = esclient.search(
    index=INDEX
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

""


## クエリを指定して検索


In [8]:
query = {
    "match": {
        "product_color": "ブラック"
    }
}

response = esclient.search(
    index=INDEX,
    query=query
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

""


## ベクトル検索（knn検索）

`text`の文字列をLLMのモデルを使って、ベクトルに変換し、knn検索を実行する。
インデックスにはすでに同じLLMのモデルを使用してドキュメントごとにベクトルが付与されているものとする。


In [18]:
import torch
import japanese_clip as ja_clip

text = "黒のリュックカバー"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device is {device}")
model, preprocess = ja_clip.load("rinna/japanese-clip-vit-b-16", cache_dir="/tmp/japanese_clip", device=device)
tokenizer = ja_clip.load_tokenizer()
encodings = ja_clip.tokenize(
    texts=text,
    device=device,
    tokenizer=tokenizer, # this is optional. if you don't pass, load tokenizer each time
)

with torch.no_grad():
    embeddings = model.get_text_features(**encodings)[0].tolist()
    

knn_query = {
    "field": "products_dense_vector",
    "query_vector": embeddings,
    "k": 10,
    "num_candidates": 100
}

response = esclient.search(
    index=INDEX,
    knn=knn_query
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

/workspace/search-research/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


""
